In [ ]:
# Imports
import pandas as pd
from selenium import webdriver
import time
import requests
import os

from pathlib import Path

# Begining timer
t0 = time.time()

# Requesting user to enter FEMA "event number"
event_number = input("Enter Event Number: ")

# Creating Directory on user's Desktop based on event number
os.mkdir(os.path.abspath(Path.home()) + f'/Desktop/{event_number}/')

# Assigning browser to be Firefox
driver = webdriver.Firefox()
# Long sleep time to account for slower loading times
time.sleep(10)

# Setting base url
base_url = (f'http://fema-cap-imagery.s3-website-us-east-1.amazonaws.com/Images/{event_number}/')

# Collecting page data
driver.get(base_url)
time.sleep(5)

# Pulling everything under the 'listing' element into a variable and converting to list
flight_list = driver.find_element_by_id('listing').text.split()

# Generating list individual flight events
# Checking against substrings and removing '/' from each itme
flight_list = [i.strip('/') for i in flight_list if ('/' in i) and ('..' not in i)]

# Function to grab image and write to flight specific directory
def image_grab(image, flight, event_number):
    image_url = f'https://s3.amazonaws.com/fema-cap-imagery/Images/{event_number}/{flight}/{image}'
    image_grab = requests.get(image_url)
    time.sleep(5)
    open(os.path.expanduser(f'~/Desktop/{event_number}/{flight}/{flight}_{image}'), 'wb').write(image_grab.content)

# Iterate through list of flights
for flight in flight_list:
    # Making directory to store images
    os.mkdir(os.path.abspath(Path.home()) + f'/Desktop/{event_number}/{flight}')
    # Loading flight event page into webdriver
    driver.get(base_url + flight + '/')
    # Pausing script to allow page to fully load
    time.sleep(10)
    # Calling list of all 
    temp_image_file_list = driver.find_element_by_id('listing').text.split()
    temp_image_file_list = [i for i in temp_image_file_list if '.jpg' in i]
    for image in temp_image_file_list:
        image_grab(image, flight, event_number)

# Closing Firefox
driver.close()

# Ending timer and printing run time
print(round((time.time() - t0), 2))